In [4]:
import collections
import helper
import numpy as np
import project_tests as tests
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [5]:
english_sentences = helper.load_data('./small_vocab_en')
french_sentences = helper.load_data('./small_vocab_fr')
print('Dataset Loaded')

Dataset Loaded


In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])
print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1622782 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [8]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [9]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')
tests.test_pad(pad)
# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [11]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
# Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)
print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [13]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
print('`logits_to_text` function loaded.')  

`logits_to_text` function loaded.


In [34]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = 'sparse_categorical_crossentropy', 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model

#tests.test_simple_model(simple_model)
tmp_x = pad(preproc_english_sentences[:len(preproc_french_sentences)], max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
#tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-2], 1))
# Train the neural network
print(len(preproc_french_sentences))
print(tmp_x.shape,preproc_french_sentences.shape)
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, epochs=10, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

114040
(114040, 21, 1) (114040, 21, 1)
Epoch 1/10
2851/2851 [==============================] - 86s 5ms/step - loss: 1.6743 - accuracy: 0.5929 - val_loss: nan - val_accuracy: 0.6379
Epoch 2/10
2851/2851 [==============================] - 13s 4ms/step - loss: 1.2219 - accuracy: 0.6504 - val_loss: nan - val_accuracy: 0.6533
Epoch 3/10
2851/2851 [==============================] - 13s 5ms/step - loss: 1.1274 - accuracy: 0.6649 - val_loss: nan - val_accuracy: 0.6683
Epoch 4/10
2851/2851 [==============================] - 13s 5ms/step - loss: 1.0679 - accuracy: 0.6818 - val_loss: nan - val_accuracy: 0.6912
Epoch 5/10
2851/2851 [==============================] - 13s 4ms/step - loss: 1.0169 - accuracy: 0.6984 - val_loss: nan - val_accuracy: 0.7106
Epoch 6/10
2851/2851 [==============================] - 13s 5ms/step - loss: 0.9839 - accuracy: 0.7073 - val_loss: nan - val_accuracy: 0.7163
Epoch 7/10
2851/2851 [==============================] - 13s 5ms/step - loss: 0.9562 - accuracy: 0.7160 - val_

In [38]:
from keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
#tests.test_embed_model(embed_model)
# tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))
tmp_x = pad(preproc_english_sentences[:len(preproc_french_sentences)], max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embeded_model.fit(tmp_x, preproc_french_sentences, epochs=10, validation_split=0.2)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
2851/2851 [==============================] - 17s 6ms/step - loss: 1.0681 - accuracy: 0.7438 - val_loss: nan - val_accuracy: 0.8463
Epoch 2/10
2851/2851 [==============================] - 15s 5ms/step - loss: 0.4256 - accuracy: 0.8713 - val_loss: nan - val_accuracy: 0.8843
Epoch 3/10
2851/2851 [==============================] - 16s 6ms/step - loss: 0.3378 - accuracy: 0.8952 - val_loss: nan - val_accuracy: 0.8982
Epoch 4/10
2851/2851 [==============================] - 15s 5ms/step - loss: 0.3005 - accuracy: 0.9053 - val_loss: nan - val_accuracy: 0.9086
Epoch 5/10
2851/2851 [==============================] - 15s 5ms/step - loss: 0.2794 - accuracy: 0.9114 - val_loss: nan - val_accuracy: 0.9128
Epoch 6/10
2851/2851 [==============================] - 15s 5ms/step - loss: 0.2635 - accuracy: 0.9157 - val_loss: nan - val_accuracy: 0.9149
Epoch 7/10
2851/2851 [==============================] - 16s 6ms/step - loss: 0.2530 - accuracy: 0.9186 - val_loss: nan - val_accuracy: 0.9178
Epoch 

In [42]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
   
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
#tests.test_bd_model(bd_model)
#tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
tmp_x = pad(preproc_english_sentences[:len(preproc_french_sentences)],  preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
bidi_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
bidi_model.fit(tmp_x, preproc_french_sentences,epochs=20, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/20
2851/2851 [==============================] - 53s 6ms/step - loss: 1.3634 - accuracy: 0.6312 - val_loss: 1.3294 - val_accuracy: 0.6170
Epoch 2/20
2851/2851 [==============================] - 17s 6ms/step - loss: 1.0422 - accuracy: 0.6802 - val_loss: 1.5290 - val_accuracy: 0.5991
Epoch 3/20
2851/2851 [==============================] - 17s 6ms/step - loss: 0.9633 - accuracy: 0.6917 - val_loss: 1.6692 - val_accuracy: 0.5981
Epoch 4/20
2851/2851 [==============================] - 17s 6ms/step - loss: 0.9192 - accuracy: 0.7002 - val_loss: 1.7777 - val_accuracy: 0.5858
Epoch 5/20
2851/2851 [==============================] - 17s 6ms/step - loss: 0.8762 - accuracy: 0.7146 - val_loss: 1.9471 - val_accuracy: 0.5776
Epoch 6/20
2851/2851 [==============================] - 17s 6ms/step - loss: 0.8336 - accuracy: 0.7285 - val_loss: 2.0563 - val_accuracy: 0.5656
Epoch 7/20
2851/2851 [==============================] - 17s 6ms/step - loss: 0.7989 - accuracy: 0.7408 - val_loss: 2.1909 - val_ac

In [44]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
#tests.test_encdec_model(encdec_model)
tmp_x = pad(preproc_english_sentences[:len(preproc_french_sentences)])
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))
encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/20
90/90 [==============================] - 6s 31ms/step - loss: 3.0735 - accuracy: 0.4305 - val_loss: 2.4956 - val_accuracy: 0.4708
Epoch 2/20
90/90 [==============================] - 2s 24ms/step - loss: 2.3479 - accuracy: 0.4936 - val_loss: 2.2555 - val_accuracy: 0.4977
Epoch 3/20
90/90 [==============================] - 2s 23ms/step - loss: 2.1938 - accuracy: 0.5044 - val_loss: 2.1099 - val_accuracy: 0.5262
Epoch 4/20
90/90 [==============================] - 2s 23ms/step - loss: 1.9898 - accuracy: 0.5265 - val_loss: 1.8866 - val_accuracy: 0.5419
Epoch 5/20
90/90 [==============================] - 2s 27ms/step - loss: 1.8179 - accuracy: 0.5501 - val_loss: 1.7487 - val_accuracy: 0.5605
Epoch 6/20
90/90 [==============================] - 3s 32ms/step - loss: 1.7047 - accuracy: 0.5678 - val_loss: 1.6558 - val_accuracy: 0.5759
Epoch 7/20
90/90 [==============================] - 2s 24ms/step - loss: 1.6244 - accuracy: 0.5753 - val_loss: 1.5862 - val_accuracy: 0.5800
Epoch 8/20
90

In [46]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
#tests.test_model_final(model_final)
print('Final Model Loaded')

Final Model Loaded


In [53]:
def final_predictions(x, y, x_tk, y_tk):
    tmp_X = pad(preproc_english_sentences[:len(preproc_french_sentences)])
    model = model_final(tmp_X.shape,
                        preproc_french_sentences.shape[1],
                        len(english_tokenizer.word_index)+1,
                        len(french_tokenizer.word_index)+1)
    
    model.fit(tmp_X, preproc_french_sentences, epochs = 2, validation_split = 0.2)
 
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    x_id_to_word = {value: key for key, value in x_tk.word_index.items()}
    x_id_to_word[0] = '<PAD>'
    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))
    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([x_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))
    #print(predictions)

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Epoch 1/2
2851/2851 [==============================] - 34s 10ms/step - loss: 0.5979 - accuracy: 0.8309 - val_loss: 0.3783 - val_accuracy: 0.8850
Epoch 2/2
1/1 [==============================] - 1s 1s/step
Sample 1:
il a vu un camion camion camion <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant cours et il est il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
june fall is sometimes quiet winter favorite france but it is yellow in freezing <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
